In [1]:
import pandas as pd 

In [2]:
import numpy as np

In [3]:
!ls ../out

output_CM_global.csv	    output_SLATM_2_global.csv	 output_SOAP_global.csv
output_FCHL_4.8_global.csv  output_SLATM_3.5_global.csv
output_FCHL_global.csv	    output_SLATM_global.csv


In [4]:
slatm_out = pd.read_csv("../out/output_FCHL_4.8_global.csv", index_col=0)

In [5]:
slatm_out

,SolN,Fragments,ObjValNoPen,ObjValWithPen
0,1,"['frag_05_c00001', 'frag_16_c00001']",1.961688,2.961688
1,2,"['frag_02_c00001', 'frag_16_c00001']",3.029463,3.029463
2,3,"['frag_02_c00001', 'frag_04_c00001', 'frag_16_...",1.036199,3.036199
3,4,"['frag_04_c00001', 'frag_16_c00001']",3.116564,3.116564
4,5,"['frag_12_c00001', 'frag_13_c00001']",1.349711,3.349711
...,...,...,...,...
995,996,"['frag_03_c00001', 'frag_04_c00001', 'frag_05_...",12.677072,14.677072
996,997,"['frag_04_c00001', 'frag_07_c00001', 'frag_09_...",8.683268,14.683268
997,998,"['frag_01_c00001', 'frag_02_c00001', 'frag_04_...",7.692075,14.692075
998,999,"['frag_01_c00001', 'frag_02_c00001', 'frag_03_...",11.699623,14.699623


In [6]:
from ast import literal_eval 

In [7]:
slatm_out["Fragments"] = slatm_out["Fragments"].apply(literal_eval)

In [8]:
# excess is missing Hs - to fix 

In [9]:
import qml

In [10]:
qm9_mol = qml.Compound("../targets/qm9_0.xyz")
qm9_ncharges = qm9_mol.nuclear_charges

In [11]:
qm9_ncharges

array([8, 6, 6, 7, 6, 8, 8, 7, 6, 1, 1])

In [12]:
def count_ncharges(list_ncharges, elements=[1,6,7,8,16]):
    ncharge_count = np.zeros(len(elements))
    for ncharge in list_ncharges:
        for i, elem in enumerate(elements):
            if ncharge == elem:
                ncharge_count[i] += 1
    return ncharge_count

In [13]:
qm9_counts = count_ncharges(qm9_ncharges)

In [14]:
qm9_counts

array([2., 4., 2., 3., 0.])

In [15]:
def get_excess_frags(frag_list):
    mols = [qml.Compound("../amons-qm9/"+x+".xyz") for x in frag_list]
    ncharges = np.concatenate([mol.nuclear_charges for mol in mols])
    ncharge_count = count_ncharges(ncharges)
    
    diff_count = ncharge_count - qm9_counts
    diff_count = np.array(diff_count).astype(int)
    excess = []
    for i, count in enumerate(diff_count): 
        ncharge = [1,6,7,8,16][i]
        if count > 0 :
            for i in range(count):
                excess.append(ncharge)
        elif count < 0:
            for i in range(abs(count)):
                excess.append(-ncharge)
    return excess

In [16]:
slatm_out["Excess"] = slatm_out["Fragments"].apply(get_excess_frags)

In [17]:
slatm_out

,SolN,Fragments,ObjValNoPen,ObjValWithPen,Excess
0,1,"[frag_05_c00001, frag_16_c00001]",1.961688,2.961688,"[1, 1, 8]"
1,2,"[frag_02_c00001, frag_16_c00001]",3.029463,3.029463,"[1, 1, 1, 1, 6, -8]"
2,3,"[frag_02_c00001, frag_04_c00001, frag_16_c00001]",1.036199,3.036199,"[1, 1, 1, 1, 1, 1, 6, 6]"
3,4,"[frag_04_c00001, frag_16_c00001]",3.116564,3.116564,"[1, 1]"
4,5,"[frag_12_c00001, frag_13_c00001]",1.349711,3.349711,"[1, 1, 1, 1, 1, 6, 7]"
...,...,...,...,...,...
995,996,"[frag_03_c00001, frag_04_c00001, frag_05_c0000...",12.677072,14.677072,"[1, 1, 1, 1, 1, 1, 1, 1, 6, 8]"
996,997,"[frag_04_c00001, frag_07_c00001, frag_09_c0000...",8.683268,14.683268,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 6, 6, 6, ..."
997,998,"[frag_01_c00001, frag_02_c00001, frag_04_c0000...",7.692075,14.692075,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
998,999,"[frag_01_c00001, frag_02_c00001, frag_03_c0000...",11.699623,14.699623,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, ..."


In [18]:
slatm_out.sort_values(by='ObjValNoPen').iloc[0:50]

,SolN,Fragments,ObjValNoPen,ObjValWithPen,Excess
2,3,"[frag_02_c00001, frag_04_c00001, frag_16_c00001]",1.036199,3.036199,"[1, 1, 1, 1, 1, 1, 6, 6]"
8,9,"[frag_01_c00001, frag_02_c00001, frag_04_c0000...",1.036199,4.036199,"[1, 1, 1, 1, 1, 1, 1, 1, 6, 6, 8]"
12,13,"[frag_01_c00001, frag_12_c00001, frag_13_c00001]",1.349711,4.349711,"[1, 1, 1, 1, 1, 1, 1, 6, 7, 8]"
4,5,"[frag_12_c00001, frag_13_c00001]",1.349711,3.349711,"[1, 1, 1, 1, 1, 6, 7]"
16,17,"[frag_01_c00001, frag_08_c00001, frag_16_c00001]",1.813027,4.813027,"[1, 1, 1, 1, 1, 6, 7, 8]"
5,6,"[frag_08_c00001, frag_16_c00001]",1.813027,3.813027,"[1, 1, 1, 6, 7]"
0,1,"[frag_05_c00001, frag_16_c00001]",1.961688,2.961688,"[1, 1, 8]"
6,7,"[frag_01_c00001, frag_05_c00001, frag_16_c00001]",1.961688,3.961688,"[1, 1, 1, 1, 8, 8]"
10,11,"[frag_03_c00001, frag_04_c00001, frag_16_c00001]",2.154426,4.154426,"[1, 1, 1, 1, 1, 6, 7]"
18,19,"[frag_01_c00001, frag_03_c00001, frag_04_c0000...",2.154426,5.154426,"[1, 1, 1, 1, 1, 1, 1, 6, 7, 8]"
